In [34]:
import os 
import json
import pandas as pd
import PIL

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

d:\DSProjects\mcqgen\mcq_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dotenv import load_dotenv
import getpass


load_dotenv()

True

In [4]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

In [6]:
llm = ChatGoogleGenerativeAI(model='gemini-pro')

In [35]:
result = llm.invoke('hello')

In [36]:
result.content

'Hello! How can I assist you today?'

In [13]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [14]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [15]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [16]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [17]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

d:\DSProjects\mcqgen\mcq_env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [18]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [19]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [20]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [21]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [22]:
file_path=r"C:\Users\krisn\Downloads\Linear algebra.txt"

In [23]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [24]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [25]:
NUMBER=3 
SUBJECT="Machine learning"
TONE="simple"

In [31]:
response=generate_evaluate_chain.invoke(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Linear algebra is a branch of mathematics, but the truth of it is that linear algebra is the
mathematics of data. Matrices and vectors are the language of data. Linear algebra is about
linear combinations. That is, using arithmetic on columns of numbers called vectors and arrays
2
1.3. Numerical Linear Algebra 3
of numbers called matrices, to create new columns and arrays of numbers. Linear algebra is the
study of lines and planes, vector spaces and mappings that are required for linear transforms.
It is a relatively young field of study, having initially been formalized in the 1800s in order
to find unknowns in systems of linear equations. A linear equation is just a series of terms and
mathematical operations where some terms are unknown; for example:
y = 4 × x + 1 (1.1)
Equations like this are linear in that they describe a line on a two-dimensional graph. The
line comes from 

In [32]:
quiz=response.get("quiz")

In [33]:
quiz

'### RESPONSE_JSON\n{"1": {"mcq": "Which of the following is NOT a component of linear algebra?", "options": {"a": "Matrices", "b": "Vectors", "c": "Trigonometry", "d": "Linear equations"}, "correct": "c"}, "2": {"mcq": "What is the purpose of linear algebra in machine learning?", "options": {"a": "To solve systems of linear equations", "b": "To make complex problems easier to understand", "c": "To create new columns and arrays of numbers", "d": "All of the above"}, "correct": "d"}, "3": {"mcq": "In the equation y = A · b, what do the symbols A and b represent?", "options": {"a": "A is the matrix of outputs, b is the vector of coefficients", "b": "A is the matrix of data, b is the vector of unknowns", "c": "A is the vector of outputs, b is the matrix of coefficients", "d": "A is the vector of data, b is the matrix of unknowns"}, "correct": "b"}}'